In [6]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import sys
import os

sys.path.append(os.path.abspath('../src'))
from vulnerability_scorer import GlobalVulnerabilityScorer
df = pd.read_csv('../data/processed/trade_data_global.csv')
tariff_master = pd.read_csv('../data/reference/trade_war_master.csv')

scorer = GlobalVulnerabilityScorer(df)
vuln_df = scorer.score_all_countries()

print("✅ Data loaded and Scorer initialized. Ready for the first visual!")

✅ Data loaded and Scorer initialized. Ready for the first visual!


In [7]:
timeline = pd.DataFrame({
    'date': ['Dec 2024','Feb 2025','Mar 2025','Apr 2025', 
             'May 2025','Aug 2025','Nov 2025','Feb 2026'],
    'avg_rate': [2.5, 10.5, 13.0, 27.0, 15.0, 18.0, 16.8, 16.8],
    'event': [
        'Pre-tariff baseline',
        'Canada/Mexico/China IEEPA tariffs begin',
        'Steel & Aluminum raised to 25%',
        'Liberation Day — US tariffs hit 100-year high',
        'US-China truce — partial reduction',
        'Country-specific reciprocal tariffs resume',
        'Xi-Trump meeting — China truce extended 1 year',
        'India secondary tariff lifted — interim deal'
    ]
})

fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    x=timeline['date'],
    y=timeline['avg_rate'],
    mode='lines+markers',
    line=dict(color='#d62728', width=3),
    marker=dict(size=12, color='#d62728'),
    hovertext=timeline['event'],
    hovertemplate='<b>%{x}</b><br>Rate: %{y}%<br>%{hovertext}<extra></extra>'
))
fig1.add_annotation(
    x='Apr 2025', y=27,
    text='<b>Liberation Day</b><br>27% — highest in 100 years',
    showarrow=True, arrowhead=2,
    bgcolor='#d62728', font=dict(color='white', size=10),
    ax=80, ay=-60
)

fig1.update_layout(
    title='<b>The US Tariff Escalation (Dec 2024 – Feb 2026)</b>',
    xaxis_title='',
    yaxis_title='Average US Tariff Rate (%)',
    height=450,
    plot_bgcolor='white',
    yaxis=dict(gridcolor='lightgray', range=[0, 35])
)

fig1.show()

if not os.path.exists('../reports'):
    os.makedirs('../reports')
fig1.write_html('../reports/viz1_escalation_timeline.html')
print("✅ Chart displayed and saved to reports/viz1_escalation_timeline.html")

✅ Chart displayed and saved to reports/viz1_escalation_timeline.html


In [8]:
import plotly.graph_objects as go
color_map = {
    '🔴 CRITICAL': '#d62728',
    '🟠 HIGH': '#ff7f0e',
    '🟡 MEDIUM': '#f9d62e',
    '🟢 LOW': '#2ca02c'
}

fig2 = go.Figure(go.Bar(
    x=vuln_df['country_name'],
    y=vuln_df['vulnerability_score'],
    marker_color=[color_map[r] for r in vuln_df['risk_level']],
    text=vuln_df['risk_level'],
    textposition='outside',
    customdata=list(zip(
        vuln_df['current_tariff'],
        vuln_df['deal_status'],
        vuln_df['notes']
    )),
    hovertemplate=(
        '<b>%{x}</b><br>'
        'Vulnerability Score: %{y}<br>'
        'Current US Tariff: %{customdata[0]}%<br>'
        'Status: %{customdata[1]}<br>'
        'Note: %{customdata[2]}<extra></extra>'
    )
))

fig2.update_layout(
    title='<b>Global Tariff Vulnerability Scores (Feb 2026)</b><br>'
          '<sup>Combined score of tariff rate, trade dependency, and deal status</sup>',
    xaxis_title='Country',
    yaxis_title='Vulnerability Score (0-100)',
    height=550,
    plot_bgcolor='white',
    yaxis=dict(gridcolor='lightgray')
)
fig2.show()
fig2.write_html('../reports/viz2_vulnerability_scores.html')

print("✅ Leaderboard displayed and saved to reports/viz2_vulnerability_scores.html")

✅ Leaderboard displayed and saved to reports/viz2_vulnerability_scores.html


In [12]:
# THE SAFETY CHECK: Run this directly before Chunk 4
import pandas as pd

# 1. Standardize names: Lowercase and no extra spaces
tariff_master.columns = tariff_master.columns.str.strip().str.lower()

# 2. If the column is just 'country', rename it to 'country_name' 
# This ensures Chunk 4 finds exactly what it's looking for.
if 'country' in tariff_master.columns and 'country_name' not in tariff_master.columns:
    tariff_master = tariff_master.rename(columns={'country': 'country_name'})

# 3. Print the results to confirm
print("✅ Column names are now ready for Chunk 4:")
print(tariff_master.columns.tolist())

✅ Column names are now ready for Chunk 4:
['country_code', 'tariff_current_feb2026', 'tariff_peak', 'deal_status', 'special_notes']


In [19]:
possible_names = ['country_name', 'Country', 'country', 'Economy', 'country_code']
actual_col = next((name for name in possible_names if name in tariff_master.columns), None)

if actual_col is None:
    print(f"❌ ERROR: I can't find a country column. Your columns are: {tariff_master.columns.tolist()}")
else:

    plot_df = tariff_master[tariff_master['country_code'] != 'USA'].copy()
    plot_df = plot_df.sort_values('tariff_peak', ascending=True)

    fig3 = go.Figure()

    # Peak tariff bar
    fig3.add_trace(go.Bar(
        name='Peak Tariff Rate',
        y=plot_df[actual_col],
        x=plot_df['tariff_peak'],
        orientation='h',
        marker_color='rgba(214, 39, 40, 0.5)',
    ))

    # Current tariff bar
    fig3.add_trace(go.Bar(
        name='Current Rate (Feb 2026)',
        y=plot_df[actual_col],
        x=plot_df['tariff_current_feb2026'],
        orientation='h',
        marker_color='rgba(44, 160, 44, 0.9)',
        text=[f"{s}" for s in plot_df['deal_status']],
        textposition='inside',
        insidetextanchor='start'
    ))

    fig3.update_layout(
        title={'text':'<b>Tariff Negotiations Tracker: Peak vs. Current Rates (Feb 2026)</b><br>'
            '<sup>Green = where rates landed after negotiations. Red = peak exposure.</sup>',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
    },
        barmode='overlay',
        xaxis_title='US Tariff Rate (%)',
        yaxis_title='',
        height=650,
        plot_bgcolor='white',
        margin=dict(t=120, b=50, l=50, r=50),
        xaxis=dict(gridcolor='lightgray'),
        yaxis=dict(automargin=True),
        legend=dict(orientation='h', 
        yanchor='bottom',
        y=1.02, 
        xanchor='right',
        x=1
    )
)
    fig3.show()
    fig3.write_html('../reports/viz3_negotiations_tracker.html')

    print("🚀 Dashboard visualization complete with full professional layout!")

🚀 Dashboard visualization complete with full professional layout!
